# Input data and parameters

In [1]:
from rdkit.Chem import rdFingerprintGenerator
from rdkit import Chem
import warnings
warnings.filterwarnings('ignore')
import math
import re
import numpy as np
import time
import csv
import pickle 

#pyqubo version==1.5.0
from pyqubo import Array
from pyqubo import Binary
#dwave-neal version==0.6.0
import neal


#SMILES-functional groups 
fgps=['*', '*C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F', '*C=C', '*C', '*CC=C', '*OC(C)=O', '*OC', '*OCOC','*OC(=O)OC', '*C(=O)OC', '*C#N', 
      '*N=C=O', '*F', '*Cl', '*Br', '*c1ccccc1', '*c1cccs1', '*C(F)(F)F', '*[Si](C)(C)C','*CCC','*NC(C)=O','*C#C','*C=CC(=O)OC']

#SMILES-skeletons 
skeletons=['*COC(*)=O','*C1OC(=O)OC1*','*C1OC(=O)C(*)OC1=O','*C1OCC2(CO1)COC(*)OC2','*C(*)(OC)OC','*COC(=O)CCC(=O)OC*', '*COC(=O)CCCC(=O)OC*',
           '*COC(=O)OC*', '*C1COC(=O)C1*', '*C1CC(=O)OC1*', '*C1CC(*)C(=O)O1', '*C1OS(=O)OC1*', '*C1CCOS(=O)C1*', '*C1COS(=O)C(*)C1', 
           '*C1CCC(*)S(=O)O1','*C1COS(=O)CC1*','*C1CCS(=O)OC1*', '*C1CCS(=O)(=O)OC1*', '*C1CC(*)OS(=O)(=O)C1', '*C1CCC(*)S(=O)(=O)O1', 
           '*C1COS(=O)(=O)CC1*', '*C1COS(=O)(=O)C(*)C1', '*C1CCOS(=O)(=O)C1*', '*C(*)C', '*CC*','*C=C*','*C(*)=C','*C1C(=O)OC(=O)C1*',
           '*C#C*','*C1CC(=O)OC(=O)C1*','*C1CC(*)C(=O)OC1=O','*C1OS(=O)(=O)C(*)S(=O)(=O)O1']

#-----set of parameters for LUMO or chemical hardeness  -------#
#clustering by lUMO value:

#chainsk=[[0, 25, 26, 28], [4, 5, 6, 7, 23, 24]]
#cycsk=[[2, 29,27, 30,31],[1, 3, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]]
#task='LUMO'

#clustering by chemical hardeness:

cycsk=[[12, 13, 14, 15, 16],[1, 2, 3, 8, 9, 10, 11, 17, 18, 19, 20, 21, 22, 27, 29, 30, 31]]
chainsk=[[0, 4, 5, 6, 7, 23, 24], [25, 26, 28]]
task='chemical_hardness'

#-----------------------------#



path = 'morgen_rad2_1024bits_'+task+'_coeffi.pkl'

with open(path, 'rb') as f:
    loaded_dict = pickle.load(f)
regT=[[],[]]
for i in range(2):
    for j in range(2):
        reg={}
        reg["coeff"]=loaded_dict["reg"+str(i)+str(j)+"_Co_In"][0]
        reg["intercept"]=loaded_dict["reg"+str(i)+str(j)+"_Co_In"][1]
        regT[i].append(reg)



#morgen fingerprint as ECFP 4
bits=1024
mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=bits)

print("---finish loading data and parameters---")

---finish loading data and parameters---


# Functions 

In [2]:

  # code to glue skeleton and functional group together
def sk_fg_glue(i_skeleton,func_group1,func_group2):
    skeleton = Chem.MolFromSmiles(i_skeleton)
    func_group1 = Chem.MolFromSmiles(func_group1)
    func_group2 = Chem.MolFromSmiles(func_group2)
    combination = Chem.CombineMols(skeleton, func_group1)
    combination = Chem.CombineMols(combination, func_group2)

    r_loc = []
    for k in range(combination.GetNumAtoms()):
        atom = combination.GetAtoms()[k].GetSymbol()
        if atom == '*':
            r_loc.append(k)
            # connect between the skeleton and functional group
    edit_mol = Chem.EditableMol(combination)
    edit_mol.AddBond(r_loc[0], r_loc[2], order=Chem.rdchem.BondType.SINGLE)
    edit_mol.AddBond(r_loc[1], r_loc[3], order=Chem.rdchem.BondType.SINGLE)
    combination = edit_mol.GetMol()
    combination_smiles = Chem.MolToSmiles(combination)
            # correct the SMLIES afther gluing
    combination_smiles = combination_smiles.replace('**', '')
    combination_smiles = combination_smiles.replace('()', '')
    if combination_smiles.count('*(*') > 0:    
      
        combination_smiles = smiles_fix(combination_smiles)
    mol = Chem.MolFromSmiles(combination_smiles)

    return  mol

def smiles_fix(smiles):
    smiles = smiles.replace('*(*', '(', 1)
    branches = []
    # find (..), (..(..)..), and (..(..)..(..)..), use & to represent a branch
    num_of_parens = len(re.findall('\(\w+\)|\(\w*\(\w+\)\w*\)|\(\w*\(\w+\)\w*\(\w+\)\w*\)', smiles))
    for i in range(num_of_parens):
        branches.append(re.search('\(\w+\)|\(\w*\(\w+\)\w*\)|\(\w*\(\w+\)\w*\(\w+\)\w*\)', smiles).group(0))
        smiles = smiles.replace(branches[i], '&', 1)
    # deal with smiles start from branch
    if smiles[0] == '&':
        smiles = re.sub('(&)([A-Za-z]\d?)', r'\2\1', smiles, 1)

    for i in range(num_of_parens):
        branches[i] = branches[i].replace('1', f'{i*2+4}')
        branches[i] = branches[i].replace('2', f'{i*2+5}')
        smiles = smiles.replace('&', branches[i], 1)
    return smiles

def code_Generator_defined(sklt,fg1,fg2,bits):
  
    #establish morgen fingerorint
   
    fg1=np.array( mfpgen.GetFingerprint (sk_fg_glue(sklt,fg1,"*"))) 
    fg2=np.array( mfpgen.GetFingerprint (sk_fg_glue(sklt,"*",fg2))) 
    sk=np.array( mfpgen.GetFingerprint (sk_fg_glue(sklt,"*","*"))) 

    #establish defined features
    code=(fg1-fg1*sk)+(fg2-fg2*sk)+sk
        
  
    x=[]

    x.append(code)
 
    return x


 #calculate the prediction results yred:prediction  
def ypred1(test,Reg_model):
    ypred=0
    
    ypred=np.matmul(Reg_model["coeff"],test) +Reg_model["intercept"]

    return ypred


# Build QUBO model

In [3]:
FGN=len(fgps)
bit=1024

#set of separated indices binary variables (SIBVs) as mentioned in paper
SK = Array.create('skeleton', shape=(len(skeletons)), vartype='BINARY')
FG= Array.create('FG', shape=(2,FGN), vartype='BINARY')

penal=0.15 #penalty coefficient, we choose the value to be 1.5*M. M denotes as the possible maximum absolute value of chemical property. The value is ~ 0.1

#constraint 1:each site can connect to only one functional group
H1=0
for j in range(2):
    h1=0
    for i in range(FGN):
        h1=h1+FG[j][i]
    H1=H1+(h1-1)**(2)
H1=penal*H1

#constraint 2:each additive can choose only one skeleton
H2=0
for i in range(len(skeletons)):
    H2=H2+SK[i]
H2=penal*(H2-1)**(2)



# objective function
obH=0
#QUBO for ring-type cluster
for i in range(len(cycsk)):
    for b in range (len(cycsk[i])):
           
       
        #weight of skeketon for QUBO term X_t               
        sk1=np.array(mfpgen.GetFingerprint (sk_fg_glue(skeletons[cycsk[i][b]],"*","*")))      
        c1=np.matmul(regT[0][i]["coeff"],sk1)+regT[0][i]["intercept"]
        obH=obH+c1*SK[cycsk[i][b]]    
        
        for j in range(len(fgps)):
            fg1=fgps[j]
            fg2=fgps[j]

                      
            # weight of a single functional group j that contributes to the skeleton t at site 1 for QUBO term X_t*X_0_j
            fg1a=np.array(mfpgen.GetFingerprint (sk_fg_glue(skeletons[cycsk[i][b]],fg1,"*")))
            fg1bit=fg1a-fg1a*sk1
            
            c1=np.matmul(regT[0][i]["coeff"],fg1bit)
            obH=obH+(c1)*SK[cycsk[i][b]]*FG[0][j]
              
            # weight of a single functional group j that contributes to the skeleton t at site 2 for QUBO term X_t*X_1_j
            fg2a=np.array(mfpgen.GetFingerprint (sk_fg_glue(skeletons[cycsk[i][b]],"*",fg2)))
            fg2bit=fg2a-fg2a*sk1

            c2=np.matmul(regT[0][i]["coeff"],fg2bit)
            obH=obH+(c2)*SK[cycsk[i][b]]*FG[1][j]
            
#QUBO for chain-type cluster         
for i in range(len(chainsk)):
    for b in range (len(chainsk[i])):
                           
        #weight of skeketon for QUBO term X_t               
        sk1=np.array(mfpgen.GetFingerprint (sk_fg_glue(skeletons[chainsk[i][b]],"*","*")))      
        c1=np.matmul(regT[1][i]["coeff"],sk1)+regT[1][i]["intercept"]
        obH=obH+c1*SK[chainsk[i][b]]    
        
        for j in range(len(fgps)):
            fg1=fgps[j]
            fg2=fgps[j]

                      
            # weight of a single functional group j that contributes to the skeleton t at site 1 for QUBO term X_t*X_0_j
            fg1a=np.array(mfpgen.GetFingerprint (sk_fg_glue(skeletons[chainsk[i][b]],fg1,"*")))
            fg1bit=fg1a-fg1a*sk1
            
            c1=np.matmul(regT[1][i]["coeff"],fg1bit)
            obH=obH+(c1)*SK[chainsk[i][b]]*FG[0][j]
              
            # weight of a single functional group j that contributes to the skeleton t at site 2 for QUBO term X_t*X_1_j
            fg2bit=np.array(mfpgen.GetFingerprint (sk_fg_glue(skeletons[chainsk[i][b]],"*",fg2)))
            fg2bit=fg2bit-fg2bit*sk1

            c2=np.matmul(regT[1][i]["coeff"],fg2bit)
            obH=obH+(c2)*SK[chainsk[i][b]]*FG[1][j]
            
#construct QUBO model with constraints and objective function 
H=obH+H1+H2            
model = H.compile()
qubo, offset = model.to_qubo(index_label=False)

# Solve by SA

In [9]:

sampler=neal.SimulatedAnnealingSampler()

ts=time.time()
sampleset1 = sampler.sample_qubo(qubo, num_reads=1000,beta_range=(1, 100), num_sweeps=100)
te=time.time()

energy10 = model.decode_sample(sampleset1.first.sample, vartype='BINARY')
print("Best solution of SA:"+str(energy10.energy))
print("Running time:"+str(te-ts))
print("----Configuration----")
for key, value in sampleset1.first.sample.items():
    if value==1:
        print(key)


Best solution of SA:0.10340270447059441
Running time:0.3233785629272461
----Config----
FG[0][16]
FG[1][16]
skeleton[28]


# Notes

In [ ]:
#optimal solution for samples: morgen_rad2_1024bits+" lumo" or "chemical_hardness"+'_coeffi.pkl'
#-----lUMO 
#Best solution:-0.09644516224565058
#----Config----
#FG[0][22]
#FG[1][22]
#skeleton[31]

#-----chemical hardness 
#Best solution:0.10340270447059441
#----Config----
#FG[0][16]
#FG[1][16]
#skeleton[28]


# Exhaustive search

In [28]:
import time
ts=time.time()

bits=1024

E=0.15 #The initial energy is updated whenever a lower energy is identified.

#calculation for ring-type cluster
for t in range(len(cycsk)):

    for a in range(len(cycsk[t])):

        linear_reg=regT[0][t]
        
        for i in range(len(fgps)):
            fg1_check=fgps[i]

            for j in range(len(fgps)):
                fg2_check=fgps[j]
                atest=code_Generator_defined(skeletons[cycsk[t][a]],fg1_check,fg2_check,bits)
                test_ypred=ypred1(atest[0],linear_reg) 
   
                if abs(E)>=abs(test_ypred):
                    E=test_ypred
                

#calculation for chain-type cluster
for t in range(len(chainsk)):
    
    for a in range(len(chainsk[t])):

        linear_reg=regT[1][t]
        for i in range(len(fgps)):
            fg1_check=fgps[i]

            for j in range(len(fgps)):
                fg2_check=fgps[j]
                atest=code_Generator_defined(skeletons[chainsk[t][a]],fg1_check,fg2_check,bits)
                test_ypred=ypred1(atest[0],linear_reg) 
                if abs(E)>=abs(test_ypred):
                    E=test_ypred
           
                 
te=time.time()
                
print("Engery of best solution:"+str(E))
print("Running time:"+str(te-ts))


Engery of best solution:0.10340270447059448
Running time:42.595603942871094
